In [1]:
import sys
sys.path.append('/workspaces/asclepius/src/Api')

print(sys.path)

['/workspaces/asclepius/src/Api/notebooks', '/usr/local/lib/python311.zip', '/usr/local/lib/python3.11', '/usr/local/lib/python3.11/lib-dynload', '', '/home/vscode/.local/lib/python3.11/site-packages', '/usr/local/lib/python3.11/site-packages', '/workspaces/asclepius/src/Api']


In [2]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.kernel import Kernel, KernelArguments
from semantic_kernel.utils.settings import (
    azure_openai_settings_from_dot_env,
)
import pandas as pd
import tiktoken

import Plugins.parse_notes_plugin as pnp

In [3]:
enc = tiktoken.get_encoding('cl100k_base')

In [4]:
note_df = pd.read_csv('/workspaces/asclepius/src/Api/data/notes_no_PHI.csv')

# calculate length of 'NOTE_TEXT' column and add value as another column
note_df['NOTE_TEXT_TOKENS'] = note_df['NOTE_TEXT'].apply(lambda x: len(enc.encode(x)))

note_df['NOTE_TEXT_TOKENS']

0    1560
1    4203
Name: NOTE_TEXT_TOKENS, dtype: int64

## These native functions use only string parsing to fetch note sections

In [23]:
plugin = pnp.ParseNotesPlugin()

note_df['CHIEF_COMPLAINT'] = note_df['NOTE_TEXT'].apply(lambda x: plugin.get_chief_complaint(input=x))
note_df['CHIEF_COMPLAINT_TOKENS'] = note_df['CHIEF_COMPLAINT'].apply(lambda x: len(enc.encode(x)))

note_df['ROS'] = note_df['NOTE_TEXT'].apply(lambda x: plugin.get_ros(input=x))
note_df['ROS_TOKENS'] = note_df['ROS'].apply(lambda x: len(enc.encode(x)))

note_df['ASSESSMENT'] = note_df['NOTE_TEXT'].apply(lambda x: plugin.get_assessment(input=x))
note_df['ASSESSMENT_TOKENS'] = note_df['ASSESSMENT'].apply(lambda x: len(enc.encode(x)))

note_df[['NOTE_TEXT_TOKENS', 'CHIEF_COMPLAINT_TOKENS', 'ROS_TOKENS', 'ASSESSMENT_TOKENS']]

#print(note_df['CHIEF_COMPLAINT'][1])


,NOTE_TEXT_TOKENS,CHIEF_COMPLAINT_TOKENS,ROS_TOKENS,ASSESSMENT_TOKENS
0,1560,481,0,447
1,4203,672,100,2624


In [ ]:
print(note_df['ASSESSMENT'][1])

## Use OpenAI to find and generate summaries of the note sections

In [7]:
deployment, api_key, endpoint = azure_openai_settings_from_dot_env()
print(endpoint)

kernel = Kernel()
service_id = "default"

kernel.add_service(AzureChatCompletion(
    service_id=service_id,    
    deployment_name=deployment,
    endpoint=endpoint,
    api_key=api_key,
))

plugin = kernel.import_plugin_from_prompt_directory('/workspaces/asclepius/src/Api/Plugins', 'summarize_note')

functions = [
    plugin["summarize_chief_complaint"], 
    plugin["summarize_lab_history"],
    plugin["summarize_assessment"]
]

for idx, note in note_df.iterrows():
    arguments = KernelArguments(input=note["NOTE_TEXT"])
    response = await kernel.invoke(functions, arguments)

    note_df.loc[idx, 'SUM_CHIEF_COMPLAINT'] = response[0].value[0].content
    note_df.loc[idx, 'SUM_LAB_HISTORY'] = response[1].value[0].content
    note_df.loc[idx, 'SUM_ASSESSMENT'] = response[2].value[0].content
    
    note_df.loc[idx, 'SUM_CHIEF_COMPLAINT_TOKENS'] = int(len(enc.encode(response[0].value[0].content)))
    note_df.loc[idx, 'SUM_LAB_HISTORY_TOKENS'] = int(len(enc.encode(response[1].value[0].content)))
    note_df.loc[idx, 'SUM_ASSESSMENT_TOKENS'] = int(len(enc.encode(response[2].value[0].content)))

cols = ['NOTE_TEXT_TOKENS', 'SUM_CHIEF_COMPLAINT_TOKENS', 'SUM_LAB_HISTORY_TOKENS', 'SUM_ASSESSMENT_TOKENS']
note_df[cols]

#chief_complaint = response[0].value[0].content


https://phjirsa-aoai.openai.azure.com/


,NOTE_TEXT_TOKENS,SUM_CHIEF_COMPLAINT_TOKENS,SUM_LAB_HISTORY_TOKENS,SUM_ASSESSMENT_TOKENS
0,1560,179.0,480.0,205.0
1,4203,226.0,323.0,315.0


In [44]:
note_df[cols].to_csv('/workspaces/asclepius/src/Api/data/output/note_summary.csv', index=False)

In [13]:
print(note_df['SUM_CHIEF_COMPLAINT'][1])

The patient's chief complaint during this visit was for an Annual Wellness Visit (Medicare) and Medication Review. The patient reported taking half of all her medications and mentioned that her smartwatch tells her she has atrial fibrillation. She also discussed her history of palpitations and cutting her medications in half, as well as her history of elevated prolactin and her use of natural supplements. The patient also mentioned her symptoms of hypothyroidism, including palpitations. The provider reviewed previous lab results and discussed adjusting the patient's medication regimen. The patient also reported occasional feelings of anger and loneliness. The provider conducted a physical exam and reviewed the patient's current medications. The patient's vital signs were within normal limits. The provider made assessments and plans for various conditions, including hypothyroidism, prolactinoma, vitamin D deficiency, essential hypertension, palpitations, and lumbar disc disease. The pro